In [ ]:
import numpy as np
from easyvec import Vec3, Mat3
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, FloatSlider
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d.proj3d import proj_transform

In [ ]:
class Target3D(object):
       
    @classmethod
    def get_simple_target(cls, pos, vel, aim, vel_aim, g=9.80665, dt=0.01, time_min=True):
        pos = cls._validate_dimension_argument(pos)
        vel = cls._validate_dimension_argument(vel)
        aim = cls._validate_dimension_argument(aim)
        vel_aim = cls._validate_dimension_argument(vel_aim)
        target = cls(pos=pos, vel=vel, aim=aim, vel_aim=vel_aim, g=g, dt=dt, time_min=time_min)
        parameters_simple_target = np.array([pos[0], pos[1], pos[2], vel[0], vel[1], vel[2], 0])
        target.set_init_cond(init_parametrs=parameters_simple_target)
        return target
    
    def __init__(self, **kwargs):
        self.g = kwargs['g']
        self.t = 0
        self.dt = kwargs['dt']
        
        self.A = kwargs['pos']
        self.D = kwargs['aim']
        self.velA = kwargs['vel'] 
        self.velD = kwargs['vel_aim']
        
        self.fly_time = 0
        self._set_fly_time(kwargs['time_min'])
        
        self.n = 7
        self.state = np.zeros(self.n)
        self.state_init = np.zeros(self.n)
    
    @property
    def pos(self):
        return self._fpos(self.t)
    
    @property
    def x(self):
        return self._fpos(self.t)[0]

    @property
    def y(self):
        return self._fpos(self.t)[1]
    
    @property
    def z(self):
        return self._fpos(self.t)[2]
    
    @property
    def vel(self):
        return self._fvel(self.t)
    
    @property
    def vel_abs(self):
        return np.sqrt(self.vel.dot(self.vel))
    
    @property
    def vel_x(self):
        return self.vel[0]
    
    @property
    def vel_y(self):
        return self.vel[1]
    
    @property
    def vel_z(self):
        return self.vel[2]
    
    def set_init_cond(self, init_parametrs=None):
        if init_parametrs is None:
            init_parametrs = self.get_random_parameters_of_target()
        self.state = np.array(init_parametrs)
        self.state_init = np.array(init_parametrs)

    def get_random_parameters_of_target(self):
        # TODO
        pass
    
    def reset(self):
        self.set_state(self.state_init)

    def set_state(self, state):
        self.state = np.array(state)

    def get_state(self):
        return self.state
    
    def get_state_init(self):
        return self.state_init
    
    def get_B(self, fly_time):
        return self.A + (self.velA * fly_time / 3)
    
    def get_C(self, fly_time): 
        return self.D - (self.velD * fly_time / 3)
    
    def get_traject(self, fly_time, vel_aim=None, n_points=100):
        vel_aim = self.velD if vel_aim == None else vel_aim
        taus = np.linspace(0, 1, n_points)
        A, velA = self.A, self.velA
        D, velD = self.D, vel_aim
        B = self.get_B(fly_time)
        C = self.get_C(fly_time)
        return np.array([
            (1-tau)**3 * A + 3*tau*(1-tau)**2 * B + 3*tau*tau*(1-tau)*C + tau**3 * D
            for tau in taus])
    
    def get_traject_vels(self, fly_time, vel_aim=None, n_points=100):
        vel_aim = self.velD if vel_aim == None else vel_aim
        taus = np.linspace(0, 1, n_points)
        A, velA = self.A, self.velA
        D, velD = self.D, vel_aim
        B = self.get_B(fly_time)
        C = self.get_C(fly_time)
        return np.array([
            (3*(1-tau)**2*(B-A) + 6*tau*(1-tau)*(C-B) + 3*tau**2*(D-C)) / fly_time
            for tau in taus
        ])
    
    def get_traject_acc(self, fly_time, vel_aim=None, n_points=100):
        vel_aim = self.velD if vel_aim == None else vel_aim
        taus = np.linspace(0, 1, n_points)
        A, velA = self.A, self.velA
        D, velD = self.D, vel_aim
        B = self.get_B(fly_time)
        C = self.get_C(fly_time)
        return np.array([
            6 * ((tau-1)*(B-A) + (1-2*tau)*(C-B) + tau*(D-C)) / (fly_time**2)
            for tau in taus
        ])
    
    def get_fly_time_minimum(self):
        # TODO
        pass
    
    def get_fly_time_random(self):
        fly_time_A = ((self.A - self.D).len() / self.velA.len())
        fly_time_D = (self.A - self.D).len() / self.velD.len()
        fly_time_min = min(fly_time_A, fly_time_D)
        fly_time_max = max(fly_time_A, fly_time_D)
        return np.random.uniform(fly_time_min, fly_time_max)
    
    def _fpos(self, t, fly_time=None):
        fly_time = fly_time if fly_time != None else self.fly_time
        A, velA = self.A, self.velA
        D, velD = self.D, self.velD
        B = self.get_B(fly_time)
        C = self.get_C(fly_time)
        tau = t / fly_time
        return (1-tau)**3 * A + 3*tau*(1-tau)**2 * B + 3*tau*tau*(1-tau)*C + tau**3 * D
    
    def _fvel(self, t, fly_time=None):
        fly_time = fly_time if fly_time != None else self.fly_time
        A, velA = self.A, self.velA
        D, velD = self.D, self.velD
        B = self.get_B(fly_time)
        C = self.get_C(fly_time)
        tau = t / fly_time
        return (3*(1-tau)**2*(B-A) + 6*tau*(1-tau)*(C-B) + 3*tau**2*(D-C)) / fly_time
    
    def _facc(self, t, fly_time=None):
        fly_time = fly_time if fly_time != None else self.fly_time
        A, velA = self.A, self.velA
        D, velD = self.D, self.velD
        B = self.get_B(fly_time)
        C = self.get_C(fly_time)
        tau = t / self.fly_time 
        return 6 * ((tau-1)*(B-A) + (1-2*tau)*(C-B) + tau*(D-C)) / (fly_time**2)
    
#     def get_amax(self, fly_time, vel_trg=None, g=Vec3(0, -9.80665, 0)):
#         vel_trg = self.velD if vel_trg == None else vel_trg
#         A, velA = self.pos, self.velA
#         D, velD = self.D, vel_trg
#         B = get_B(fly_time)
#         C = get_C(fly_time)
#         a1 = (C - B) * 6 - (B - A) * 6 - g
#         a2 = (D - C) * 6 - (C - B) * 6 - g
#         return np.fmax(a1.len(), a2.len()) / (fly_time**2)

#       cpdef (double, double) get_max_v_a(double delta_t, Vec2 A, Vec2 B, Vec2 C, Vec2 D, int n=33, int rounds=3, bint inc_g=False):
#     cdef Vec2 BA = B.sub_vec(A)
#     cdef Vec2 CB = C.sub_vec(B)
#     cdef Vec2 DC = D.sub_vec(C)
#     cdef double min_v = 1e13
#     cdef double max_v = 0
#     cdef int i, j, imax
#     cdef double dt = 1.0 / (n-1)
#     cdef double t, vel_len, t0, t1
#     t0 = 0.0
#     t1 = 1.0
#     for j in range(rounds):
#         dt = (t1-t0) / (n-1)
#         for i in range(n):
#             t = t0 + i * dt
#             vel_len = (BA.mul_num(3*(1-t)*(1-t)/delta_t).add_vec(CB.mul_num(6*t*(1-t)/delta_t)).add_vec(DC.mul_num(3*t*t/delta_t))).len() 
#             if vel_len > max_v:
#                 max_v = vel_len
#                 imax = i
#         t = t0 + imax * dt
#         t0 = t - dt
#         t1 = t + dt
#         if t0 < 0:
#             t0 = 0.0
#         if t1 > 1:
#             t1 = 1.0
#     cdef Vec2 g = Vec2(0, 0)
#     if inc_g:
#         g.y = -9.81
#     cdef Vec2 a1 = ((CB.mul_num(6)).sub_vec(BA.mul_num(6))).sub_vec(g)
#     cdef Vec2 a2 = ((DC.mul_num(6)).sub_vec(CB.mul_num(6))).sub_vec(g)
#     return max_v, fmax(a1.len(), a2.len())/delta_t/delta_t

#     cpdef (double, double) get_min_max_v(double delta_t, Vec2 A, Vec2 B, Vec2 C, Vec2 D, int n=42):
#     cdef Vec2 BA = B.sub_vec(A)
#     cdef Vec2 CB = C.sub_vec(B)
#     cdef Vec2 DC = D.sub_vec(C)
#     cdef double min_v = 1e13
#     cdef double max_v = 0
#     cdef int i
#     cdef double dt = 1.0 / (n-1)
#     cdef double t, vel_len
#     for i in range(n):
#         t = i * dt
#         vel_len = (BA.mul_num(3*(1-t)*(1-t)/delta_t).add_vec(CB.mul_num(6*t*(1-t)/delta_t)).add_vec(DC.mul_num(3*t*t/delta_t))).len() 
#         # print(t, vel_len)
#         if vel_len < min_v:
#             min_v = vel_len
#         if vel_len > max_v:
#             max_v = vel_len
#     return min_v, max_v
    
    def to_dict(self):
        return { 
            't': self.t,
            'x': self.x,
            'y': self.y,
            'z': self.z,
            'v': self.vel_abs,
            'vx': self.vel[0],
            'vy': self.vel[1],
            'vz': self.vel[2]
        }
    
    def _set_fly_time(self, time_min=True):
        self.fly_time = self.get_fly_time_minimum() if time_min else self.get_fly_time_random()
    
    @staticmethod
    def _validate_dimension_argument(array, n=3):
        if len(array) == 0:
            return Vec3(0,0,0)            
        elif len(array) == n:
            try:
                l = [float(elem) for elem in array]
            except ValueError:
                raise ValueError("Один или несколько элементов в <{!r}> не действительное(-ые) число(-а)".format(seq))
            else:
                arg1, arg2, arg3 = l
                return Vec3(arg1, arg2, arg3)  
        else:
            raise ValueError("Неожиданное число элементов. Получено: {}, Ожидалось: {}.".format(len(array), n))  

In [ ]:
target = Target3D.get_simple_target(pos=Vec3(5e3, 1e3, 300),
                                    vel=Vec3(-100, -100, -10),
                                    aim=Vec3(100,100,0),
                                    vel_aim=Vec3(-100,-10,0))

In [ ]:
def get_traject(target, fly_time):
    return target.get_traject(fly_time)

In [ ]:
def get_traject_vels(target, fly_time):
    return target.get_traject_vels(fly_time)

In [ ]:
def get_traject_acc(target, fly_time):
    return target.get_traject_acc(fly_time)

In [ ]:
trg_pos = (500, 500, 500)
aim_pos = (-10, -10, 0)
fly_times = np.linspace(20, 80, 3)
t = 15

In [ ]:
v0x, v0y, v0z = -10, -10, 10
v1x, v1y, v1z = -15, -20, -5

In [ ]:
%matplotlib

In [ ]:
class Arrow3D(FancyArrowPatch):

    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0, 0), (0, 0), *args, **kwargs)
        self._xyz = (x, y, z)
        self._dxdydz = (dx, dy, dz)

    def draw(self, renderer):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        super().draw(renderer)
        
def _arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)


setattr(Axes3D, 'arrow3D', _arrow3D)

In [ ]:
fig = plt.figure(dpi=120)
ax = plt.axes(projection='3d')
    
target = Target3D.get_simple_target(pos=trg_pos,
                                    vel=(v0x, v0y, v0z),
                                    aim=aim_pos,
                                    vel_aim=(v1x, v1y, v1z))
    
for fly_time in fly_times:
    traject = get_traject(target, fly_time)
    ax.plot(traject[:,0], traject[:,1], traject[:,2], label=f'{fly_time} c')
    point_pos = target._fpos(t, fly_time)
    point_vel = target._fvel(t, fly_time)
    ax.scatter(point_pos[0], point_pos[1], point_pos[2])
    ax.arrow3D(
    point_pos[0], point_pos[1], point_pos[2], 
    3*point_vel[0], 3*point_vel[1], 3*point_vel[2],
    mutation_scale=5,
    fc='blue',
    label='$t=0$'
    )    
ax.arrow3D(
    trg_pos[0], trg_pos[1], trg_pos[2], 
    3*v0x, 3*v0y, 3*v0z,
    mutation_scale=5,
    fc='blue',
    label='$t=0$'
    )
ax.arrow3D(
    aim_pos[0], aim_pos[1], aim_pos[2], 
    3*v1x, 3*v1y, 3*v1z,
    mutation_scale=5,
    fc='green',
    label='$t=t_1$'
)    

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title('Траектории цели при fly time')
ax.view_init(elev=30, azim=-150)
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(dpi=120)
ax = plt.axes(projection='3d')
for fly_time in fly_times:
    traject_vels = get_traject_vels(target, fly_time)
    ax.plot(traject_vels[:,0], traject_vels[:,1], traject_vels[:,2], label=f'{fly_time} c')
ax.set_xlabel('vx')
ax.set_ylabel('vy')
ax.set_zlabel('vz')
ax.set_title('Годограф вектора скорости при различных fly time')
ax.view_init(elev=30, azim=-150)
ax.legend()
plt.show()

In [ ]:
fig = plt.figure(dpi=120)
ax = plt.axes(projection='3d')
for fly_time in fly_times:
    traject_acc = get_traject_acc(target, fly_time)
    ax.plot(traject_acc[:,0], traject_acc[:,1], traject_acc[:,2], label=f'{fly_time} c')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title('Годограф вектора ускорения при различных fly time')
ax.view_init(elev=30, azim=-150)
ax.legend()
plt.show()

In [ ]:
%matplotlib inline

In [ ]:
trg_pos = (100, 100, 100)
aim_pos = (-10, -10, 0)
fly_times = np.linspace(20, 100, 3)
t = 15
@interact(v0x=FloatSlider(min=-100, max=100, step=1, value=-5), 
          v0y=FloatSlider(min=-100, max=100, step=1, value=-1),
          v0z=FloatSlider(min=-100, max=100, step=1, value=-1),
          v1x=FloatSlider(min=-100, max=100, step=1, value=-3), 
          v1y=FloatSlider(min=-100, max=100, step=1, value=-5),
          v1z=FloatSlider(min=-100, max=100, step=1, value=-2))
def plot_sever(v0x, v0y, v0z, v1x, v1y, v1z):
    
    fig = plt.figure(dpi=150)
    ax = plt.axes(projection='3d')
    
    target = Target3D.get_simple_target(pos=trg_pos,
                                        vel=(v0x, v0y, v0z),
                                        aim=aim_pos,
                                        vel_aim=(v1x, v1y, v1z))
    
    for fly_time in fly_times:
        traject = get_traject(target, fly_time)
        ax.plot(traject[:,0], traject[:,1], traject[:,2], label=f'{fly_time} c')
        point_pos = target._fpos(t, fly_time)
        point_vel = target._fvel(t, fly_time)
        ax.scatter(point_pos[0], point_pos[1], point_pos[2])
        ax.arrow3D(
        point_pos[0], point_pos[1], point_pos[2], 
        3*point_vel[0], 3*point_vel[1], 3*point_vel[2],
        mutation_scale=5,
        fc='blue',
        label='$t=0$'
        )   
        
    ax.arrow3D(
        trg_pos[0], trg_pos[1], trg_pos[2], 
        3*v0x, 3*v0y, 3*v0z,
        mutation_scale=5,
        fc='blue',
        label='$t=0$'
    )
    ax.arrow3D(
        aim_pos[0], aim_pos[1], aim_pos[2], 
        3*v1x, 3*v1y, 3*v1z,
        mutation_scale=5,
        fc='green',
        label='$t=t_1$'
    )    

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.set_title('Траектории цели при fly time')
#     ax.view_init(elev=30, azim=-150)
    ax.legend()
    plt.tight_layout()
    plt.show()